In [1]:
import matplotlib.pyplot as plt
import torch
from data_loader import test_dataset
# from models.ResNet import get_resnet18_model
from models.EfficientNet import get_efficientnet_b2_model

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_efficientnet_b2_model(num_classes=2)
model.load_state_dict(torch.load("saved_models/efficient_net_b2.pth", map_location=device))
model.to(device)
model.eval()


Training set size: 960
Test set size: 240


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [3]:
import numpy as np
from sklearn.metrics import confusion_matrix


# Collect all probabilities and true labels from the test set
all_probs = []
all_labels = []

from torch.utils.data import DataLoader
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)[:, 1]  # Probability for class 1 (unhealthy)
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_probs = np.array(all_probs)
all_labels = np.array(all_labels)

def compute_sens_spec(threshold):
    preds = (all_probs > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(all_labels, preds, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return sensitivity, specificity


In [4]:
from sklearn.metrics import roc_auc_score, auc

auc = roc_auc_score(all_labels, all_probs)
print(f"AUC: {auc:.3f}")

AUC: 0.975


In [15]:
threshold = 0.17 # Try different values here!
sens, spec = compute_sens_spec(threshold)
print(f"Threshold: {threshold:.2f} | Sensitivity: {sens:.3f} | Specificity: {spec:.3f}")

from sklearn.metrics import roc_auc_score, auc

auc = roc_auc_score(all_labels, all_probs)
print(f"AUC: {auc:.3f}")

Threshold: 0.17 | Sensitivity: 0.940 | Specificity: 0.893
AUC: 0.975


In [ ]:
import torch
import time
from torch.utils.data import DataLoader

# Set up device and model (already done in your notebook)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# Create a DataLoader for exactly 100 images
test_loader_100 = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4, pin_memory=True)

# Get one batch of 100 images
inputs, labels = next(iter(test_loader_100))
inputs = inputs.to(device)

with torch.no_grad():
    torch.cuda.synchronize()  # Ensure all previous CUDA ops are done
    start_time = time.time()
    outputs = model(inputs)
    torch.cuda.synchronize()  # Wait for inference to finish
    end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time for 100 images: {inference_time:.4f} seconds")
print(f"Average time per image: {inference_time/100:.6f} seconds")

In [ ]:
# create a confusion matrix
cm = confusion_matrix(all_labels, (all_probs > threshold).astype(int), labels=[0, 1])
print("Confusion Matrix:")
print(cm)

In [ ]:
from seaborn import heatmap
import seaborn as sns
# print confusion matrix
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['Non-referable DR', 'Referable DR']); ax.yaxis.set_ticklabels(['Non-referable DR', 'Referable DR']);

In [ ]:
# Unnormalize for visualization
def unnormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

img_np = unnormalize(img_tensor[0].cpu()).permute(1, 2, 0).numpy()
plt.imshow(img_np)
plt.title(f"True: {label.item()}, Pred: {predicted_class}")
plt.axis('off')
plt.show()
